# Predição de mortalidade em internações por pneumonias bacterianas sensíveis à Atenção Primária no Brasil, 2017-2021

## Análise Exploratória

### 1. Importando dados

In [40]:
# IMPORTANDO PACOTES 
import numpy as np
import pandas as pd   

In [41]:
#IMPORTANDO CSV 
colunas =  ["MORTE", "CID_MORTE", "DIAG_PRINC", "DT_INTER", "ANO_CMPT", "MES_CMPT", "DIAG_SECUN",  "IDADE", "SEXO", "UF_ZI", "DIAS_PERM", "NUM_FILHOS", "INSTRU", "MARCA_UTI", "INFEHOSP"]

sih_pneumo = pd.read_csv("sih_pneumonia_1721.csv")
sih_pneumo.set_axis(colunas, axis= 1, inplace= True)
sih_pneumo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734686 entries, 0 to 734685
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   MORTE       734686 non-null  int64  
 1   CID_MORTE   734686 non-null  int64  
 2   DIAG_PRINC  734686 non-null  object 
 3   DT_INTER    734686 non-null  int64  
 4   ANO_CMPT    734686 non-null  int64  
 5   MES_CMPT    734686 non-null  int64  
 6   DIAG_SECUN  734686 non-null  int64  
 7   IDADE       734686 non-null  int64  
 8   SEXO        734686 non-null  int64  
 9   UF_ZI       734686 non-null  int64  
 10  DIAS_PERM   734686 non-null  int64  
 11  NUM_FILHOS  734686 non-null  int64  
 12  INSTRU      734686 non-null  int64  
 13  MARCA_UTI   734686 non-null  int64  
 14  INFEHOSP    0 non-null       float64
dtypes: float64(1), int64(13), object(1)
memory usage: 84.1+ MB


### 2. Preparando os dados

In [46]:
sih_pneumo['DT_INTER'] = sih_pneumo['DT_INTER'].astype("string")
sih_pneumo['ANO_INT'] = sih_pneumo['DT_INTER'].str.slice(0, 4)
sih_pneumo['ANO_INT'].value_counts()

0         2017
1         2016
2         2016
3         2017
4         2016
          ... 
734681    2022
734682    2022
734683    2022
734684    2022
734685    2022
Name: DT_INTER, Length: 734686, dtype: string

In [43]:
sih_pneumo['DT_INTER']


TypeError: object of type 'int' has no len()

In [ ]:
sih_pneumo['ANO_INT'] = sih_pneumo['DT_INTER'].str.split(0, 3)
sih_pneumo['ANO_CMPT'].value_counts()

#### 2.1 Identificando e classificando os dados

#### 2.2 *Missing values*

#### 2.3 *Outliers*

## 3. Análise descritiva 

#### 3.1 Medidas resumo 

#### 3.2 Análise de correlação e associação das variáveis